In [1]:
# 가상환경 생성 (Python 3.10 또는 3.11)

# ----- terminal에서 실행 -----
# py -3.10 -m venv agent-env                                  <- venv 생성
# Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass  <- PS 보안 비활성화
# .\Task_recommend\use_agent\agent-env\Scripts\activate       <- venv 활성화

In [ ]:
# %pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.12.12-cp310-cp310-win_amd64.whl.metadata (7.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached duckduckgo_search-8.0.3-py3-none-any.whl.metadata (16 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached frozenlist-1.7.0-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached greenlet-3.2.3-cp310-cp310-win_amd64.whl.metadata (4.2 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using 

  You can safely remove it manually.


In [1]:
# from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from jinja2 import Template
from langchain.agents import AgentExecutor, create_react_agent

from dotenv import load_dotenv
import json
import os
import re
import json
import random

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
JSON_FILE_PATH = "..\\use_openai\\data\\id_added_dummy_data.json"
# RCMD_PROMPT_PATH = "..\\prompts\\agent_recommend.md"
RCMD_PROMPT_PATH = "..\\prompts\\react_prompt.md"  # ReAct에 맞는 프롬프트로 수정
FEATURE_ID_PATH = "featureId.txt"

In [4]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()
    
# template = Template(RCMD_PROMPT)  # template 객체 생성

# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))  # test

# Read featureId keys
with open(FEATURE_ID_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    feature_id_list = [line.strip() for line in lines]
    # print(feature_id)  # test

<class 'list'>


In [7]:
# ##### test #####
# FEATURE_ID = int(random.choice(feature_id_list))
# print(FEATURE_ID, type(FEATURE_ID))

In [5]:
# featureId로 해당 category, feature 이하의 actions만 추출
FEATURE_ID = int(random.choice(feature_id_list))

def extract_workspace_view(input_data, id):
    # featureId로 작업목록 + workspaceId 가져오기
    for item in input_data:
        for category in item.get("recommendedCategories", []):
            for feature in category.get("features", []):
                if feature.get("featureId") == id:
                    return {
                        "workspaceId": item["workspaceId"],
                        "recommendedCategories": [
                            {
                                "categoryId": category["categoryId"],
                                "name": category["name"],
                                "features": [feature]
                            }
                        ]
                    }

workspace_data = extract_workspace_view(data, FEATURE_ID)
workspace_data
# print(json.dumps(result, ensure_ascii=False, indent=2))

if workspace_data:
    print(json.dumps(workspace_data, ensure_ascii=False, indent=2))
else:
    print("해당 featureId를 찾을 수 없습니다.")

{
  "workspaceId": 1,
  "recommendedCategories": [
    {
      "categoryId": 4,
      "name": "게시판 기능",
      "features": [
        {
          "featureId": 11,
          "name": "게시글 관리",
          "actions": [
            {
              "name": "조회수 관리",
              "importance": 5,
              "startDate": "2025-06-15T14:25:34.094027",
              "endDate": "2025-06-25T14:25:34.094027"
            },
            {
              "name": "파일 첨부",
              "importance": 1,
              "startDate": "2025-06-16T14:25:34.094031",
              "endDate": "2025-06-25T14:25:34.094031"
            },
            {
              "name": "게시글 검색",
              "importance": 4,
              "startDate": "2025-06-13T14:25:34.094034",
              "endDate": "2025-06-15T14:25:34.094034"
            },
            {
              "name": "게시글 신고",
              "importance": 5,
              "startDate": "2025-06-16T14:25:34.094036",
              "endDate": "2025-06-22T14:25:34.

In [9]:
workspace_data = extract_workspace_view(data, FEATURE_ID)

template = Template(RCMD_PROMPT)                              # 프롬프트 template 객체 생성

# 템플릿&작업목록 데이터 결합
# formatted_input = template.render(wrapped_data=workspace_data)  # 문자열 생성. template 내 변수({})들을 실제값으로 치환
# formatted_input = template.render(wrapped_data=json.dumps(workspace_data, ensure_ascii=False))   # 이거나
formatted_input = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))  # 이거 시도해보시면 됨

print(formatted_input)  # test
print(type(formatted_input))

TemplateSyntaxError: expected token 'end of print statement', got ':'

In [ ]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [ ]:
# 변수{} 삽입 가능한 템플릿 생성
prompt = PromptTemplate(
    input_variables=["featureNname", "existingActions"],
    template="""
    다음은 '{featureName}' 기능의 기존 작업들입니다:

    {existingActions}

    이 작업의 다음 단계로서, 자연스럽게 이어질 수 있는 작업 3개를 추천하세요.
    각 작업은 다음 항목을 포함해야 합니다:

    - name: 구체적 작업명
    - startDate: YYYY-MM-DDTHH:MM:SS 형식
    - endDate: YYYY-MM-DDTHH:MM:SS 형식
    - importance: 1~5 정수(값이 클수록 중요한 작업임)

    형식은 반드시 JSON 배열 형태로 출력하세요.
    예시:
    ```json
    [
      {
        "name": "...",
        "startDate": "...",
        "endDate": "...",
        "importance": 3
      },
      ...
    ]
    ```
    """
)

# llm(OpenAI) <-> Langchain 연결
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
##### test #####
search = DuckDuckGoSearchRun()
conv_data = json.dumps(workspace_data, ensure_ascii=False)  # dict -> str
pattern = r'({[\s\S]*})'
match = re.search(pattern, conv_data, re.DOTALL)
json_block = match.group(0)
print(type(json_block))
print(json_block)

NameError: name 'formatted_input' is not defined

In [ ]:
# LLM으로 작업 추천 받는 함수
def generate_next_actions(input_dict: dict) -> str:
    return llm_chain.run(input_dict)

# 웹 검색 툴 정의
search = DuckDuckGoSearchRun()

# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수
def parse_and_generate_all(input_str: dict) -> str:

    # conv_data = json.dumps(input_str, ensure_ascii=False)  # dict -> str
    # pattern = r'({[\s\S]*?"categoryId"\s*:\s*\d+[\s\S]*?})'
    # match = re.search(pattern, conv_data, re.DOTALL)
    data = input_str
    if not match:
        raise ValueError("** Can not find JSON blocks **")

    # json_block = match.group(0)  # 정규식에 매치된 문자열
    data = json.loads(json_block)
    # data = input_str

    for category in data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서
            # llm_input = {                                   # features, actions 가져오기
            #     "featureName": feature["name"],
            #     "existingActions": [a["name"] for a in feature.get("actions", [])]
            # }

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"(웹 검색 실패: {e})"

            # LLM 입력 구성 (웹검색 결과 포함)
            llm_input = {
                "featureName": feature["name"],
                "existingActions": [a["name"] for a in feature.get("actions", [])],
                "webSearch": search_result[:500]  # 길이 제한
            }

            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 0
                })

    return json.dumps(data, ensure_ascii=False, indent=2)


In [ ]:
type(workspace_data)  # test
# workspace_data

dict

In [ ]:
parse_and_generate_all(workspace_data)  # test

NameError: name 'match' is not defined

In [ ]:
# 툴 정의
parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업(actions) 3개를 추가합니다."
)

final_tool = Tool(
    name="Final Answer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다."
)

tools = [parse_tool, final_tool]

In [ ]:
# Agent
# agent_executor = initialize_agent(
#     tools=[parse_tool, final_tool],
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     handle_parsing_errors=True,
#     prompt=formatted_input
# )

# 프롬프트 객체 생성
prompt = PromptTemplate(
    template=RCMD_PROMPT,
    input_variables=["workspace_data", "tools", "tool_names", "agent_scratched"]
)

# 먼저 에이전트 생성하고
# create_react_agent()는 initialize_agent() 보다 세밀한 설정 가능
agent = create_react_agent(
    llm=llm,
    tools=tools,  # 단순 tools 선언, AgentExecutor에서 tools 재전달 필요
    prompt=prompt
)

# 다음에 AgentExecutor로 루프 반복 컨트롤?
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,  # tools의 기능 전달
    max_iterations=5,
    max_execution_time=30,
    early_stopping_method="generate",
    verbose=True
)

NameError: name 'llm' is not defined

In [ ]:
prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))
response = agent_executor.invoke({"input": prompt_filled})

# *** 답변 생성 확인 ***
# print(response)



> Entering new AgentExecutor chain...


KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "workspaceId"\'}.  Expected: [\'\\n  "workspaceId"\', \'agent_scratchpad\', \'input\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\']\nNote: if you intended {\n  "workspaceId"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "workspaceId"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
response

{'input': 'Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:\n\n{\n  "workspaceId": "작업 중인 workspaceId",\n  "recommendedCategories": [\n    {\n      "categoryId": "작업 중인 categoryId",\n      "name": "카테고리 이름",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 정수,\n      "features": [\n        {\n          "featureId": "작업 중인 featureId",\n          "name": "기능 이름",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "importance": 정수,\n          "actions": [\n            {\n              "name": "추천 action",\n              "startDate": "YYYY-MM-DDTHH:MM:SS",\n              "endDate": "YYYY-MM-DDTHH:MM:SS",\n              "importance": 정수\n            },\n            {\n              "name": "추천 action",\n         